<a href="https://colab.research.google.com/github/mlangsman/fastai-experiments/blob/main/Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 Classifier Experiment

## 📁 Install dependencies

We need the Fastai library for model training and DuckDuckGo search to retrieve images for our dataset.


In [3]:
!pip install -Uqq fastai ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 99.4 MB/s eta 0:00:00


## Download images


In [12]:
from ddgs import DDGS
from fastcore.all import *

def search_images(keywords, max_images=10):
  results = DDGS().images(keywords, max_results=max_images) #run a search
  imageUrls = L(results).itemgot('image') #extract just image urls
  return imageUrls

search_images('car')

(#10) ['https://images.pexels.com/photos/3729464/pexels-photo-3729464.jpeg?cs=srgb&dl=pexels-mikebirdy-3729464.jpg&fm=jpg','https://images.pexels.com/photos/210019/pexels-photo-210019.jpeg?cs=srgb&dl=pexels-pixabay-210019.jpg&fm=jpg','https://img2.thejournal.ie/article/3119677/river?version=3119704&width=1340','https://i.pinimg.com/originals/45/74/c6/4574c68d814dc0d3a64611ee401ea7dc.jpg','https://images.pexels.com/photos/1545743/pexels-photo-1545743.jpeg?cs=srgb&dl=pexels-mayday-1545743.jpg&fm=jpg','https://www.vervemagazine.co.nz/wp-content/uploads/2024/03/Hero-Car.jpg','https://wallpaperbat.com/img/104795-free-download-download-wallpaper-beautiful-car-gallery-1680x1050.jpg','http://thewowstyle.com/wp-content/uploads/2015/04/car-hot.jpg','https://cdn.vox-cdn.com/thumbor/IZ7fpJNSeEO1v2vNapVlLYlCWzc=/214x0:1037x549/1200x800/filters:focal(214x0:1037x549)/cdn.vox-cdn.com/uploads/chorus_image/image/45200072/new-ford-gt-supercar-0006.0.0.jpg','https://www.carscoops.com/wp-content/uploads/20